In [2]:
import os 
from dotenv import load_dotenv
from langchain_core.messages import SystemMessage, ToolMessage, BaseMessage, HumanMessage
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from langchain_core.tools import tool
from typing import TypedDict, Annotated, Sequence
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

In [3]:
load_dotenv(r"D:\PYTHON\Agentic AI\.ENV FILES\.env")

True

In [4]:
GROQ_Key = os.getenv("GROQ_API_KEY")

In [5]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]

In [6]:
@tool
def Add(a: int , b: int):
    """This is a addition function that add the numbers"""
    
    return a + b

In [7]:
Tools = [Add]

In [8]:
Model = ChatGroq(model="llama-3.1-8b-instant", api_key=GROQ_Key).bind_tools(Tools)

In [9]:
def Call_model(state: AgentState) -> AgentState:
    system_prompt = SystemMessage(content="You are an AI assistant. Answer questions clearly and precisely.")
    messages = [system_prompt] + state["messages"]
    response = Model.invoke(messages)
    return {"messages": state["messages"] + [response]}

In [10]:
def Continue(state: AgentState):
    last_message = state["messages"][-1]
    if hasattr(last_message, "tool_calls") and last_message.tool_calls:
        return "continue"
    else:
        return "end"

In [11]:
Graph = StateGraph(AgentState)
Graph.add_node("Agent", Call_model)
Tool = ToolNode(tools=Tools)
Graph.add_node("Tool", Tool)
Graph.set_entry_point("Agent")

Graph.add_conditional_edges(
    "Agent",
    Continue,
    {
        "continue": "Tool",
        "end": END,
    },
)

Graph.add_edge("Tool", "Agent")

Compiled_Graph = Graph.compile()


In [13]:
result = Compiled_Graph.invoke({
    "messages": [HumanMessage(content="Add 23 and 98")]
})

result

{'messages': [HumanMessage(content='Add 23 and 98', additional_kwargs={}, response_metadata={}, id='565bc819-b330-41b2-963f-665cc18376b3'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'wma5e6t47', 'function': {'arguments': '{"a":23,"b":98}', 'name': 'Add'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 242, 'total_tokens': 261, 'completion_time': 0.034386839, 'prompt_time': 0.014769709, 'queue_time': 0.044793161, 'total_time': 0.049156548}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ab04adca7d', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--fa3af7e8-cd65-49b8-a197-82530114c34b-0', tool_calls=[{'name': 'Add', 'args': {'a': 23, 'b': 98}, 'id': 'wma5e6t47', 'type': 'tool_call'}], usage_metadata={'input_tokens': 242, 'output_tokens': 19, 'total_tokens': 261}),
  ToolMessage(content='121', name='Add', id='1f7bb61e-04bf-466d-bee2-e64ffe3ffd77', tool_call_i